In [36]:
from find_source import summary, significant
from astropy.io import fits
from astropy.coordinates import Angle, SkyCoord
import astropy.units as u
import glob

In [37]:
summary('../data/11151/3c279.fits', True, False, False)

{'int_peak_val': 12.789649963378906,
 'int_peak_coord': (-0.6042424478692535, -0.6042424478692535),
 'int_snr': 1033.4329542488892,
 'calc_int_snr': 746.9733913272844,
 'int_prob': 0.0,
 'calc_int_prob': 0.0,
 'ext_peak_val': [0.11062327027320862, 0.10841279476881027],
 'ext_peak_coord': [(0.6042424478692535, -22.356970571162382),
  (-1.8127273436077607, 21.148485675423874)],
 'ext_snr': [8.156838745138412, 8.35947913680472],
 'calc_ext_snr': 2.801546810997779,
 'ext_prob': [3.1678594828139107e-12, 8.328408101379607e-13],
 'calc_ext_prob': 1.000000000000001,
 'field_center': (0, 0),
 'rms': 0.012375887483358383,
 'calc_rms_val': 0.017121961922436344,
 'n_incl_meas': 26.018807354386986,
 'n_excl_meas': 393.24977269309557,
 'radius': [15.716911040408949, 7.442086890547327, 7.442086890547327]}

In [38]:
summary('../data/sdssj152527.48+050029.9.fits', True, False, False)

{'int_peak_val': 0.00460889283567667,
 'int_peak_coord': (4.157787343125113, 1.2599355585227614),
 'int_snr': 3.938832132401772,
 'calc_int_snr': 3.484304729783657,
 'int_prob': 0.007722816896333792,
 'calc_int_prob': 0.04547026085659896,
 'ext_peak_val': 'No significant external peak',
 'ext_peak_coord': 'No significant external peak',
 'ext_snr': 'No significant external peak',
 'calc_ext_snr': 3.525667445039651,
 'ext_prob': 'No significant external peak',
 'calc_ext_prob': 1.0000000000000024,
 'field_center': (0, 0),
 'rms': 0.0011701165931299329,
 'calc_rms_val': 0.0013227582525374695,
 'n_incl_meas': 184.30750354148685,
 'n_excl_meas': 4734.657446796816,
 'radius': [46.02584220100205]}

In [39]:
def get_info_for_catalog(fits_file: str):

    summ = summary(fits_file, True, False, False)

    header_data = fits.getheader(fits_file)
    ctype1 = header_data['CTYPE1']
    crval1 = header_data['CRVAL1']
    cunit1 = header_data['CUNIT1']
    cdelt1 = header_data['CDELT1']
    ctype2 = header_data['CTYPE2']
    crval2 = header_data['CRVAL2']
    cunit2 = header_data['CUNIT2']
    cdelt2 = header_data['CDELT2']
    name = header_data['OBJECT']

    interesting_sources = [name]

    n_ext_sources = 0
    if type(summ['ext_peak_val']) == list:
        n_ext_sources += len(summ['ext_peak_val'])

    ra_index = 0
    dec_index = 1

    if 'RA' in ctype1:
        ra = crval1
    elif 'RA' in ctype2:
        ra = crval2
        ra_index = 1
    else:
        raise ValueError('No RA in image')

    if 'DEC' in ctype1:
        dec = crval1
        dec_index = 0
    elif 'DEC' in ctype2:
        dec = crval2
    else:
        raise ValueError('No dec in image')

    if cunit1 != cunit2:
        raise ValueError('Axes have different units')

    center = SkyCoord(ra, dec, unit=cunit1)

    for i in range(n_ext_sources):
        ext_info = {}
        ext_info['flux_density'] = summ['ext_peak_val'][i]
        ext_info['error'] = summ['rms'] #not sure about this one

        ext_ra_offset = summ['ext_peak_coord'][i][ra_index] * u.arcsec
        ext_dec_offset = summ['ext_peak_coord'][i][dec_index] * u.arcsec
        ext_info['coord'] = center.spherical_offsets_by(ext_ra_offset, ext_dec_offset)

        ext_info['internal'] = False

        interesting_sources.append(ext_info)

    if significant(fits_file):
        int_info = {}
        int_info['flux_density'] = summ['int_peak_val']
        int_info['error'] = summ['rms'] #not sure about this one

        int_ra_offset = summ['int_peak_coord'][ra_index] * u.arcsec
        int_dec_offset = summ['int_peak_coord'][dec_index] * u.arcsec
        int_info['coord'] = center.spherical_offsets_by(int_ra_offset, int_dec_offset)

        int_info['internal'] = True

        interesting_sources.append(int_info)

    return interesting_sources

In [40]:
get_info_for_catalog('../data/11151/3c279.fits')

['3c279',
 {'flux_density': 0.11062327027320862,
  'error': 0.012375887483358383,
  'coord': <SkyCoord (ICRS): (ra, dec) in deg
      (194.04669371, -5.79552249)>,
  'internal': False},
 {'flux_density': 0.10841279476881027,
  'error': 0.012375887483358383,
  'coord': <SkyCoord (ICRS): (ra, dec) in deg
      (194.04601889, -5.78343764)>,
  'internal': False},
 {'flux_density': 12.789649963378906,
  'error': 0.012375887483358383,
  'coord': <SkyCoord (ICRS): (ra, dec) in deg
      (194.04635629, -5.78948007)>,
  'internal': True}]

In [41]:
get_info_for_catalog('../data/1407+2827.fits')

['1407+2827',
 {'flux_density': 0.004832044243812561,
  'error': 0.0010278000263497233,
  'coord': <SkyCoord (ICRS): (ra, dec) in deg
      (211.75152378, 28.45387035)>,
  'internal': True}]